In [2]:
import math
import os

import numpy as np
import pandas as pd
from scipy.signal import savgol_filter

In [39]:
#band_pass_filtering
from scipy.signal import cheby1, filtfilt


def band_pass_filtering(data, fs, filter_type):
    if filter_type == "bcg":
        [b_cheby_high, a_cheby_high] = cheby1(2, 0.5, [2.5 / (fs / 2)], btype='high', analog=False)
        bcg_ = filtfilt(b_cheby_high, a_cheby_high, data)
        [b_cheby_low, a_cheby_low] = cheby1(4, 0.5, [5.0 / (fs / 2)], btype='low', analog=False)
        filtered_data = filtfilt(b_cheby_low, a_cheby_low, bcg_)
    elif filter_type == "breath":
        [b_cheby_high, a_cheby_high] = cheby1(2, 0.5, [0.01 / (fs / 2)], btype='high', analog=False)
        bcg_ = filtfilt(b_cheby_high, a_cheby_high, data)
        [b_cheby_low, a_cheby_low] = cheby1(4, 0.5, [0.4 / (fs / 2)], btype='low', analog=False)
        filtered_data = filtfilt(b_cheby_low, a_cheby_low, bcg_)
    else:
        filtered_data = data
    return filtered_data

In [40]:
#detect_peaks
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt


def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising',
                 kpsh=False, valley=False, show=False, ax=None):
    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan - 1, indnan + 1))), invert=True)]
        # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size - 1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind] - x[ind - 1], x[ind] - x[ind + 1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                              & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indices by their occurrence
        ind = np.sort(ind[~idel])

    if show:
        if indnan.size:
            x[indnan] = np.nan
        if valley:
            x = -x
        _plot(x, mph, mpd, threshold, edge, valley, ax, ind)

    return ind


def _plot(x, mph, mpd, threshold, edge, valley, ax, ind):
    """Plot results of the detect_peaks function, see its help."""
    try:
        import matplotlib.pyplot as plt
    except ImportError:
        print('matplotlib is not available.')
    else:
        if ax is None:
            _, ax = plt.subplots(1, 1, figsize=(8, 4))

        ax.plot(x, 'b', lw=1)
        if ind.size:
            label = 'valley' if valley else 'peak'
            label = label + 's' if ind.size > 1 else label
            ax.plot(ind, x[ind], '+', mfc=None, mec='r', mew=2, ms=8,
                    label='%d %s' % (ind.size, label))
            ax.legend(loc='best', framealpha=.5, numpoints=1)
        ax.set_xlim(-.02 * x.size, x.size * 1.02 - 1)
        ymin, ymax = x[np.isfinite(x)].min(), x[np.isfinite(x)].max()
        yrange = ymax - ymin if ymax > ymin else 1
        ax.set_ylim(ymin - 0.1 * yrange, ymax + 0.1 * yrange)
        ax.set_xlabel('Data #', fontsize=14)
        ax.set_ylabel('Amplitude', fontsize=14)
        mode = 'Valley detection' if valley else 'Peak detection'
        ax.set_title("%s (mph=%s, mpd=%d, threshold=%s, edge='%s')"
                     % (mode, str(mph), mpd, str(threshold), edge))
        # plt.grid()
        plt.show()

In [41]:
#beat_to_beat
import numpy as np

#from detect_peaks import detect_peaks


def compute_rate(beats, time, mpd):

    indices = detect_peaks(beats, mpd=mpd)

    if len(indices) > 1:
        peak_to_peak = []
        for i in range(0, indices.size - 1):
            peak_to_peak = np.append(peak_to_peak, time[indices[i + 1]] - time[indices[i]])
        mean_heart_rate = np.average(peak_to_peak, axis=0)
        bpm_avg = 1000 * (60 / mean_heart_rate)
        return np.round(bpm_avg, decimals=2), indices
    else:
        return 0.0, 0.0

In [42]:
#compute-vitals
import numpy as np

#from beat_to_beat import compute_rate


def vitals(t1, t2, win_size, window_limit, sig, time, mpd, plot=0):
    all_rate = []
    for j in range(0, window_limit):
        sub_signal = sig[t1:t2]
        [rate, indices] = compute_rate(sub_signal, time, mpd)
        all_rate.append(rate)
        t1 = t2
        t2 += win_size
    all_rate = np.vstack(all_rate).flatten()
    return all_rate

In [43]:
#detect_apnea

import math

import numpy as np
import pandas as pd


def unix_time_converter(unix_time):
    tm = pd.to_datetime(unix_time, unit='ms')
    readable_time = tm.tz_localize('UTC').tz_convert('Asia/Singapore').strftime("%H.%M.%S")
    return readable_time


def apnea_events(data, utc_time, thresh):
    pt1, pt2, win_size = 0, 1500, 1500
    hop_size, win_shift = 500, win_size
    limit = int(math.floor(data.size / win_size))
    counter = 0
    start_time, stop_time, apnea_events = [], [], {}
    for i in range(0, limit):

        StDs = []
        sub_data = data[pt1:pt2]
        sub_utc_time = utc_time[pt1:pt2]
        sub_sub_utc_time = []

        for so in range(0, win_shift, hop_size):
            ndx = np.arange(so, so + hop_size)
            sub_sub_utc_time.append(sub_utc_time[ndx])
            fiber_optic_data = sub_data[ndx]

            StDs.append(np.std(fiber_optic_data, ddof=1))

        T = np.mean(StDs)

        ind = [i for i, v in enumerate(StDs) if v <= thresh * T]

        if ind:
            for j in ind:
                counter += 1
                current_time = sub_sub_utc_time[j]
                start_time.append(unix_time_converter(current_time[0]))
                stop_time.append(unix_time_converter(current_time[-1]))
                print('\nApnea Information')
                print('start time : ', start_time, ' stop time : ', stop_time)

        pt1 = pt2
        pt2 += win_size

    apnea_events[0] = start_time
    apnea_events[1] = stop_time
    return apnea_events



In [44]:
#detect_body_movement

import math
import matplotlib
matplotlib.use('agg')
from matplotlib.pyplot import plot, savefig, figure
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np

# The segmentation is performed based on the standard deviation of each time window
# In general if the std is less than 15, it means tha the there is no any pressure applied to the mat.
# if the std if above 2 * MAD all time-windows SD it means, we are facing body movements.
# On the other hand, if the std is between 15 and 2 * MAD of all time-windows SD,
# there will be a uniform pressure to the mat. Then, we can analyze the sleep patterns

def detect_patterns(pt1, pt2, win_size, data, time, plot):
    # store start and end time point
    pt1_ = pt1
    pt2_ = pt2

    limit = int(math.floor(data.size / win_size))
    flag = np.zeros([data.size, 1])
    event_flags = np.zeros([limit, 1])

    segments_sd = []

    for i in range(0, limit):
        sub_data = data[pt1:pt2]
        segments_sd.append(np.std(sub_data, ddof=1))
        pt1 = pt2
        pt2 += win_size

    mad = np.sum(np.abs(segments_sd - np.mean(segments_sd, axis=0))) / (1.0 * len(segments_sd))

    thresh1, thresh2 = 15, 2 * mad

    pt1, pt2 = pt1_, pt2_

    for j in range(0, limit):
        std_fos = np.around(segments_sd[j])
        if std_fos < thresh1:  # No-movement
            flag[pt1:pt2] = 3
            event_flags[j] = 3
        elif std_fos > thresh2:  # Movement
            flag[pt1:pt2] = 2
            event_flags[j] = 2
        else:
            flag[pt1:pt2] = 1  # Sleeping
            event_flags[j] = 1
        pt1 = pt2
        pt2 += win_size

    pt1, pt2 = pt1_, pt2_

    # Function to highlight activities
    data_for_plot = data
    width = np.min(data_for_plot)
    if width < 0:
        height = np.max(data_for_plot) + np.abs(width)
    else:
        height = np.max(data_for_plot)

    if plot == 1:
        # fig = plt.figure()
        current_axis = plt.gca()
        plt.plot(np.arange(0, data.size), data_for_plot, '-k', linewidth=1)
        plt.xlabel('Time [Samples]')
        plt.ylabel('Amplitude [mV]')
        plt.gcf().autofmt_xdate()

    for j in range(0, limit):
        sub_data = data_for_plot[pt1:pt2]
        sub_time = np.arange(pt1, pt2)/50
        if event_flags[j] == 3:  # No-movement
            if plot == 1:
                plt.plot(sub_time, sub_data, '-k', linewidth=1)
                current_axis.add_patch(
                    Rectangle((pt1, width), win_size, height, facecolor="#FAF0BE", alpha=.2))
        elif event_flags[j] == 2:  # Movement
            if plot == 1:
                plt.plot(sub_time, sub_data, '-k', linewidth=1)
                current_axis.add_patch(
                    Rectangle((pt1, width), win_size, height, facecolor="#FF004F", alpha=1.0))
        else:  # Sleeping
            if plot == 1:
                plt.plot(sub_time, sub_data, '-k', linewidth=1)
                current_axis.add_patch(
                    Rectangle((pt1, width), win_size, height, facecolor="#00FFFF", alpha=.2))
        pt1 = pt2
        pt2 += win_size

    plt.savefig('rawData.png')
    #plt.savefig('../results/rawData.png')

    # Remove Body Movements and bed-empty activities
    ind2remove = np.sort(np.append(np.where(event_flags == 3), np.where(event_flags == 2)), axis=None)
    mask = np.ones(data.size, dtype=bool)
    mask[ind2remove] = False
    filtered_data = data[mask]
    filtered_time = time[mask]

    return filtered_data, filtered_time

In [45]:
#note savefig above

In [46]:
#modwt_matlab_fft

import math
import sys

import numpy as np
import pyfftw
import pywt


def modwt(x, wname, J):
    # % Convert data to row vector
    if x.shape[0] > 1:
        x = x.flatten()

    # % Record original data length
    datalength = x.size

    # % Check that the level of the transform does not exceed floor(log2(len(x))
    Jmax = np.floor(math.log(datalength, 2))
    if J <= 0 or J > Jmax:
        print('Wavelet:modwt:MRALevel')
        sys.exit()

    # % obtain new signal length if needed
    siglen = x.size
    Nrep = siglen

    # % Scale the scaling and wavelet filters for the MODWT
    wavelet = pywt.Wavelet(wname)
    Lo = wavelet.rec_lo
    Hi = wavelet.rec_hi
    Lo = np.array(Lo) / np.sqrt(2)
    Hi = np.array(Hi) / np.sqrt(2)

    # % Ensure Lo and Hi are row vectors
    if Lo.shape[0] > 1:
        Lo = Lo.flatten()
    if Hi.shape[0] > 1:
        Hi = Hi.flatten()

    # % If the signal length is less than the filter length, need to
    # % periodize the signal in order to use the DFT algorithm
    if siglen < len(Lo):
        xp = np.tile(x, (1, len(Lo) - siglen))
        x = np.append(x, xp)
        Nrep = x.size

    # % Allocate coefficient array
    w = []  # np.zeros(shape=(J + 1, Nrep))

    # % Obtain the DFT of the filters
    G = pyfftw.interfaces.numpy_fft.fft(Lo, Nrep, planner_effort='FFTW_ESTIMATE', threads=1).T
    H = pyfftw.interfaces.numpy_fft.fft(Hi, Nrep, planner_effort='FFTW_ESTIMATE', threads=1).T

    # %Obtain the DFT of the data
    Vhat = pyfftw.interfaces.numpy_fft.fft(x, planner_effort='FFTW_ESTIMATE').T

    # % [Vhat,What] = modwtfft(X,G,H,J)
    def modwtdec(X, G, H, J):
        N = X.size
        upfactor = 2 ** J
        Gup = G[np.mod(upfactor * np.arange(0, N), N)]
        Hup = H[np.mod(upfactor * np.arange(0, N), N)]
        Vhat = np.multiply(Gup, X)
        What = np.multiply(Hup, X)
        return Vhat, What

    # % Main MODWT algorithm
    for jj in range(J):
        [Vhat, What] = modwtdec(Vhat, G, H, jj)
        w.append(pyfftw.interfaces.numpy_fft.ifft(What, planner_effort='FFTW_ESTIMATE', threads=1).real)
    w.append(pyfftw.interfaces.numpy_fft.ifft(Vhat, planner_effort='FFTW_ESTIMATE', threads=1).real)
    w = np.vstack(w)
    w = w[:, 0:siglen]
    return w



In [47]:
#modwt_mra_matlab_fft

import sys

import numpy as np
import pyfftw
import pywt


def modwtmra(w, wname):
    # % The input to modwtmra must be a matrix
    if w.shape[0] == 1 and w.shape[1] > 1 or w.shape[0] > 1 and w.shape[1] == 1:
        print('Wavelet:modwt:MRASize')
        sys.exit()

    # % get the size of the output coefficients
    cfslength = w.shape[1]
    J0 = w.shape[0] - 1

    nullinput = np.zeros(cfslength)
    N = cfslength

    # % Scale the scaling and wavelet filters for the MODWT
    wavelet = pywt.Wavelet(wname)
    Lo = wavelet.rec_lo
    Hi = wavelet.rec_hi
    Lo = np.array(Lo) / np.sqrt(2)
    Hi = np.array(Hi) / np.sqrt(2)

    if cfslength < len(Lo):
        wp = np.tile(w, (1, len(Lo) - cfslength))
        w = np.append(w, wp, axis=1)
        cfslength = w.shape[1]
        nullinput = np.zeros(cfslength)

    G = pyfftw.interfaces.numpy_fft.fft(Lo, cfslength, planner_effort='FFTW_ESTIMATE', threads=1).T
    H = pyfftw.interfaces.numpy_fft.fft(Hi, cfslength, planner_effort='FFTW_ESTIMATE', threads=1).T

    # % Allocate array for MRA
    mra = []  # np.zeros(shape=(J0 + 1, N))

    def imodwtrec(Vin, Win, G, H, J):
        N = Vin.size
        Vhat = pyfftw.interfaces.numpy_fft.fft(Vin, planner_effort='FFTW_ESTIMATE', threads=1).T
        What = pyfftw.interfaces.numpy_fft.fft(Win, planner_effort='FFTW_ESTIMATE', threads=1).T
        upfactor = 2 ** J
        Gup = np.conj(G[np.mod(upfactor * np.arange(0, N), N)])
        Hup = np.conj(H[np.mod(upfactor * np.arange(0, N), N)])
        Vout = pyfftw.interfaces.numpy_fft.ifft(np.multiply(Gup, Vhat) + np.multiply(Hup, What),
                                                planner_effort='FFTW_ESTIMATE', threads=1).real
        return Vout

    def imodwtDetails(coefs, nullinput, lev, Lo, Hi, N):
        v = nullinput
        w = coefs
        for jj in range(lev + 1, 0, -1):
            vout = imodwtrec(v, w, Lo, Hi, jj - 1)
            w = nullinput
            v = vout
        details = v[0:N]
        return details

    def imodwtSmooth(scalingcoefs, nullinput, Lo, Hi, N, J0):
        v = scalingcoefs
        for J in range(J0 + 1, 0, -1):
            vout = imodwtrec(v, nullinput, Lo, Hi, J - 1)
            v = vout
        smooth = v[0:N]
        return smooth

    # % Main MRA - MODWT algorithm
    for J in range(J0, 0, -1):
        wcfs = w[J - 1]
        details = imodwtDetails(wcfs, nullinput, J - 1, G, H, cfslength)
        details = details[0:N]
        mra.append(details)
    scalingcoefs = w[J0:]
    smooth = imodwtSmooth(scalingcoefs.flatten(), nullinput, G, H, cfslength, J0 - 1)
    mra = mra[::-1]
    mra.append(smooth[0: N])
    mra = np.vstack(mra)
    return mra


In [48]:
#data_subplot

import matplotlib
matplotlib.use('agg')
from matplotlib.pyplot import plot, savefig, figure
import matplotlib.pyplot as plt
import numpy as np
import os

def normalize(sig):
    sig = np.divide(sig, np.sum(np.abs(sig) ** 2, axis=-1) ** (1. / 2))
    return sig


def data_subplot(raw_data, movement, breathing, dc, t1, t2):
    raw_data = normalize(raw_data)
    movement = normalize(movement)
    breathing = normalize(breathing)
    dc = normalize(dc)
    steps = np.arange(t1, t2) / 50

    fig = plt.figure()
    ax1 = fig.add_subplot(3, 1, 1)
    ax2 = fig.add_subplot(3, 1, 2)
    ax3 = fig.add_subplot(3, 1, 3)

    ax1.plot(steps, raw_data[t1:t2], lw=2, color='k', label='Raw Signal')
    ax1.set_xlabel('Time [Seconds]')
    ax1.set_ylabel('Ampltiude')
    ax1.legend(bbox_to_anchor=(1, 1.3), loc='center right')
    plt.subplots_adjust(hspace=0.8)

    ax2.plot(steps, movement[t1:t2], lw=2, color='k', label='BCG Signal')
    ax2.plot(steps, dc[t1:t2], lw=2, color='r', ls='-.', label='Level 4 Smooth')
    ax2.set_xlabel('Time [Seconds]')
    ax2.set_ylabel('Ampltiude')
    ax2.legend(bbox_to_anchor=(1, 1.3), loc='center right')
    plt.subplots_adjust(hspace=0.8)

    ax3.plot(steps, breathing[t1:t2], lw=2, color='k', label='Respiratory Signal')
    ax3.set_xlabel('Time [Seconds]')
    ax3.set_ylabel('Ampltiude')
    ax3.legend(bbox_to_anchor=(1, 1.3), loc='center right')
    plt.subplots_adjust(hspace=0.8)
    
    #plt.savefig('../results/vitals.png')
    plt.savefig('vitals.png', dpi = 300)

In [49]:
#remove_nonlnear_trend

import numpy as np


def remove_nonLinear_trend(input_signal, order):
    # Detrend with a n order polynomial
    model = np.polyfit(np.arange(0, input_signal.size), input_signal, order)
    predicted = np.polyval(model, np.arange(0, input_signal.size))
    filteredSignal = input_signal - predicted
    return filteredSignal



In [50]:
import os
os.getcwd()

'/Users/juwonlo/My python stuff/Python Implementation of MODWT  with Sample Signal'

In [51]:
# Main program starts here
print('\nstart processing ...')

file = 'sample_data.csv'

if file.endswith(".csv"):
    fileName = os.path.join(file)
    if os.stat(fileName).st_size != 0:
        rawData = pd.read_csv(fileName, sep=",", header=None, skiprows=1).values
        utc_time = rawData[:, 0]
        data_stream = rawData[:, 1]

        start_point, end_point, window_shift, fs = 0, 500, 500, 50
        # 
        data_stream, utc_time = detect_patterns(start_point, end_point, window_shift, data_stream, utc_time, plot=1)
        # 
        # BCG signal extraction
        movement = band_pass_filtering(data_stream, fs, "bcg")
        # 
        # Respiratory signal extraction
        breathing = band_pass_filtering(data_stream, fs, "breath")
        breathing = remove_nonLinear_trend(breathing, 3)
        breathing = savgol_filter(breathing, 11, 3)
        # 
        w = modwt(movement, 'bior3.9', 4)
        dc = modwtmra(w, 'bior3.9')
        wavelet_cycle = dc[4]
        # 
        # Vital Signs estimation - (10 seconds window is an optimal size for vital signs measurement)
        t1, t2, window_length, window_shift = 0, 500, 500, 500
        hop_size = math.floor((window_length - 1) / 2)
        limit = int(math.floor(breathing.size / window_shift))
        # 
         # Heart Rate
        beats = vitals(t1, t2, window_shift, limit, wavelet_cycle, utc_time, mpd=1, plot=0)
        print('\nHeart Rate Information')
        print('Minimum pulse : ', np.around(np.min(beats)))
        print('Maximum pulse : ', np.around(np.max(beats)))
        print('Average pulse : ', np.around(np.mean(beats)))
        # Breathing Rate
        beats = vitals(t1, t2, window_shift, limit, breathing, utc_time, mpd=1, plot=0)
        print('\nRespiratory Rate Information')
        print('Minimum breathing : ', np.around(np.min(beats)))
        print('Maximum breathing : ', np.around(np.max(beats)))
        print('Average breathing : ', np.around(np.mean(beats)))
        # 
        thresh = 0.3
        events = apnea_events(breathing, utc_time, thresh=thresh)
        # 
         # Plot Vitals Example
        t1, t2 = 2500, 2500 * 2
        data_subplot(data_stream, movement, breathing, wavelet_cycle, t1, t2)
        # 
        print('\nEnd processing ...')
    # 


start processing ...

Heart Rate Information
Minimum pulse :  65.0
Maximum pulse :  109.0
Average pulse :  99.0

Respiratory Rate Information
Minimum breathing :  10.0
Maximum breathing :  23.0
Average breathing :  17.0

Apnea Information
start time :  ['10.00.17']  stop time :  ['10.00.27']

End processing ...


In [1]:
pwd

'/Users/juwonlo/My python stuff/Python Implementation of MODWT  with Sample Signal'